In [1]:
# import required module
from pathlib import Path
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf
import numpy as np
import csv


In [3]:

def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]



In [4]:

def extractInfos(f) :
  comment,repository,commit,commitType="","","",""

  with open(file, "r",encoding="utf-8", errors="ignore") as f:
        
        for i, line in enumerate(f):

          #extract the commit information from the first line
          if i==0:
            infos=line.split(',')
            repository=infos[0]
            commit=infos[4]
            commitType=infos[3]
            comment= infos[-1:][0]
          else:
            if ("diff --git" in line) or ("Signed-off-by" in line) or ("Reported-by" in line) or ("Acked-by" in line):
              break
            else:
              if line !="\n":
                comment += line
  return (repository,commit,commitType,comment)


In [5]:
# Init the model
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")


In [7]:

# assign directory
directory = 'D:\PFEMaster\Dataset et pretraitement\Dataset_VCCPlus'


In [8]:
 # iterate over files in that directory
files = Path(directory).glob('*.txt')

In [9]:
#initialization of variables
iter=0
code=""
newFileName=""
newFileLine=0
repository=""
commit=""
commitType=""
comment =""
tableRef=[]


In [11]:
# iterate the commit files to extract the modified files
for file in files:
        repository,commit,commitType,comment= extractInfos(file)
        with open(file, "r",encoding="utf-8", errors="ignore") as f:
            for i, line in enumerate(f):
                  #new modified file by the commit
                  if "diff --git" in line:
                        if newFileName != "" and code!= "" :
                              extension = newFileName[-3:][:2]

                              #do not include .h files
                              if (extension != ".h"):

                                    # Tokenization
                                    nl_tokens=tokenizer.tokenize(comment)
                                    code_tokens=tokenizer.tokenize(code)

                                    tokens=nl_tokens+[tokenizer.sep_token]+code_tokens

                                    #check that the series does not exceed 510
                                    if len(tokens) > 510:

                                      fractions = list(divide_chunks(code_tokens,509-len(nl_tokens)))
                                      for fract in fractions:
                                        fract= [tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+fract+[tokenizer.sep_token]
                                        tokens_ids=tokenizer.convert_tokens_to_ids(fract)
                                        tableRef.append((f.name,repository,commit,commitType,newFileLine,newFileName[:-1],tokens_ids))

                                    else:
                                      tokens= [tokenizer.cls_token]+tokens+[tokenizer.sep_token]
                                      # Convert tokens to ids
                                      tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
                                      tableRef.append((f.name,repository,commit,commitType,newFileLine,newFileName[:-1],tokens_ids))
                                    


                        newFileName=line.rsplit('/', 1)[1]
                        newFileLine=i
                        code=""

                  else:
                    code+=line


            extension = newFileName[-3:][:2]
            if (extension != ".h" and code!= ""):

                  # Tokenization
                  nl_tokens=tokenizer.tokenize(comment)
                  code_tokens=tokenizer.tokenize(code)

                  tokens=nl_tokens+[tokenizer.sep_token]+code_tokens

                  #check that the series does not exceed 510

                  if len(tokens) > 510:

                          fractions = list(divide_chunks(code_tokens,509-len(nl_tokens)))
                          for fract in fractions:
                            fract= [tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+fract+[tokenizer.sep_token]
                            tokens_ids=tokenizer.convert_tokens_to_ids(fract)
                            tableRef.append((f.name,repository,commit,commitType,newFileLine,newFileName[:-1],tokens_ids))
                  else:

                          tokens= [tokenizer.cls_token]+tokens+[tokenizer.sep_token]

                          # Convert tokens to ids
                          tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
                          #embeddings.append(tokens_ids)
                          tableRef.append((f.name,repository,commit,commitType,newFileLine,newFileName[:-1],tokens_ids))


            newFileName=""
            newFileLine=0
            code=""



In [15]:
with open('D:/Tokens/CodeBert/CodeBert_tableTokens_files.csv', "w") as f:
    wr = csv.writer(f)
    wr.writerows(tableRef)

In [ ]:

#calculate the maximum length of the series of tokens generated from the inputs
max_value=max([len(serieTokens[6]) for serieTokens in tableRef ]) 

with open("D:/Embeddings/CodeBert/CodeBert_filesEmbeddings.csv", "w") as f:
    for serieTokens in tableRef :
        
          #Unify the length of the series of tokens
          tokens_same_size=np.pad(serieTokens[6], (0, max_value-len(serieTokens[6])), 'constant')

          context_embeddings=model(torch.tensor(tokens_same_size).unsqueeze(0))
          t = torch.flatten(context_embeddings.pooler_output)
          arr=torch.Tensor.numpy(t.detach()) #the embedding vector
         
          wr = csv.writer(f) #save the vector in a file
          wr.writerow(arr)
          

